In [1]:
import re
import os
import pandas as pd

# from google.colab import drive  # Colab 드라이브 마운트
# drive.mount('/content/drive', force_remount=True)

BASE_PATH    = ''  # 드라이브 내 작업 경로
IN_PATH = os.path.join(BASE_PATH,"여성맞춤정책_요약_2차_결과_요약수정_최종.xlsx")

def fix_bullet_lines(text: str) -> str:
    lines = text.splitlines()
    fixed = []
    for line in lines:
        m = re.match(r'^(\s*)-(\s*)(.*)$', line)
        if m:
            content = m.group(3)
            fixed.append(f" - {content.strip()}")
        else:
            fixed.append(line)
    return "\n".join(fixed)

def fix_parentheses_spacing(text: str) -> str:
    s = text

    # --- [A] 숫자만 들어있는 괄호는 플레이스홀더로 대체 (내부는 붙여쓰기 유지) ---
    placeholder_fmt = "<<<P_NUM:{}>>>"
    nums = []

    def _num_hold(m):
        num = m.group(1)
        idx = len(nums)
        nums.append(num)
        return placeholder_fmt.format(idx)

    s = re.sub(r'\(\s*(\d+)\s*\)', _num_hold, s)

    # --- [B] 일반 괄호 내부 공백 정리: "(내용)" -> "( 내용 )" ---
    s = re.sub(r'\(\s*([^\)]*?)\s*\)', r'( \1 )', s)

    # --- [C] 괄호 외부 공백 정리(일반 괄호에만 적용됨) ---
    s = re.sub(r'(?P<prev>\S)\(', r'\g<prev> (', s)  # 앞쪽
    s = re.sub(r'\)(?P<next>\S)', r') \g<next>', s)  # 뒤쪽

    # --- [D] 숫자 괄호 복원 ---
    def _num_restore(m):
        idx = int(m.group(1))
        return f'({nums[idx]})'
    s = re.sub(r'<<<P_NUM:(\d+)>>>', _num_restore, s)

    # --- [E] 숫자 괄호의 외부 공백 보정 (복원 후 적용) ---
    # 앞쪽: 붙은문자(숫자/한글/영문/%) + "(숫자)" -> "문자 (숫자)"
    s = re.sub(r'(?P<prev>\S)\((\d+)\)', r'\g<prev> (\2)', s)
    # 뒤쪽: "(숫자)문자" -> "(숫자) 문자"
    s = re.sub(r'\((\d+)\)(?P<next>\S)', r'(\1) \g<next>', s)

    # --- [F] 라인별 다중 공백 축소(선행 들여쓰기는 유지) ---
    lines = []
    for line in s.splitlines():
        line = line.replace('\t', ' ')
        m = re.match(r'^(\s*)(.*)$', line)
        lead, body = m.group(1), m.group(2)
        body = re.sub(r' {2,}', ' ', body)
        lines.append(lead + body)
    return "\n".join(lines)

def process_cell(val):
    if pd.isna(val):
        return val
    t = str(val)
    t = fix_bullet_lines(t)
    t = fix_parentheses_spacing(t)
    return t

# 모든 시트/모든 열 처리
xls = pd.read_excel(IN_PATH, sheet_name=None)
out_sheets = {}
for sheet_name, df in xls.items():
    df2 = df.copy()
    for col in df2.columns:
        if df2[col].dtype == object:
            df2[col] = df2[col].map(process_cell)
    out_sheets[sheet_name] = df2

# Google Drive 경로로 저장
OUT_PATH = os.path.join(BASE_PATH, "여성맞춤정책_요약_2차_결과_정리.xlsx")

with pd.ExcelWriter(OUT_PATH, engine="xlsxwriter") as writer:
    for sheet_name, df in out_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

df.head(3)

,대상유형,지역,제목,detail_url,신청기간,신청방법,접수기관,지원형태,지원대상,지원내용,문의처,기타,카테고리_분류,지원형태_분류,지원대상_원문,지원대상_초벌요약,지원내용_원문,지원내용_초벌요약
0,한부모,강원특별자치도,강릉의료원 보건의료 복지 통합지원 서비스,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/O000...,상시신청,○ 기타\n - 전자공문,NaN,"현금 ( 감면 ) , 서비스 ( 의료 )","지원대상\n○ 국민기초생활수급자 ( 생계, 의료, 주거, 교육 급여 )\n\n○ 차...",지원내용\n○ 의료비 지원\n - 질병치료와 관련한 강릉의료원 의료서비스 이용시 급...,보건의료복지통합지원팀 ( ☎033-610-1492 ),NaN,취약계층,"직접금전지원,감면할인지원,의료지원,서비스","(1) 국민기초생활수급자 ( 생계, 의료, 주거, 교육 급여 )\n(2) 차상위계층...","국민기초생활수급자 ( 생계, 의료, 주거, 교육 급여 ) 차상위계층 가구건강보험료 ...",(1) 의료비 지원\n - 질병치료와 관련한 강릉의료원 의료서비스 이용시 급여 본인...,의료비 및 만성질환 관리 및 모니터링
1,한부모,강원특별자치도,강원형 수선유지 주거급여 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/6420...,접수기관 별 상이,○ 방문 신청\n - 주민센터 : 주소지 읍면동 주민센터에 방문 신청\n - 시군 ...,"주민센터,시·군·구청",현물,"지원대상\n○ 기준 중위소득 50%이하 차상위계층 중 고령자, 장애인, 소년·소녀가...","지원내용\n○ 지원대상\n - 기준 중위소득 50%이하 차상위계층 중 고령자, 장애...",건축과 ( ☎033-249-3464 ),NaN,"한부모,고령자,취약계층","주거지원,물품지원","(1) 기준 중위소득 50%이하 차상위계층 중 고령자, 장애인, 소년·소녀가정, 다...","기준 중위소득 50%이하 차상위계층 중 고령자, 장애인, 소년·소녀가정, 다자녀 가...","(1) 해당사항\n - 기준 중위소득 50%이하 차상위계층 중 고령자, 장애인, 소...",가구당 최대 5백만원 지원 등
2,한부모,강원특별자치도,방학 중 아동급식 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/6530...,상시신청,거주지 관할 읍/면/동 행정복지센터에 방문 신청,주민센터,현물,지원대상\n(1) 아래의 어느 하나에 해당하는 아동으로서 결식우려가 있는 아동\n ...,"지원내용\n○ 지원연령: 18세 미만의 취학 아동 ( 초등학교, 중학교, 고등학교 ...",복지정책과 ( ☎033-249-2267 ),NaN,"한부모,임신/출산/육아,초/중/고등학생,취약계층","교육지원,물품지원",(1) 아래의 어느 하나에 해당하는 아동으로서 결식우려가 있는 아동\n - 국민기초...,"국민기초생활보장법 제2조제2호에 따른 수급자 가구의 아동, 차상위계층 아동 또는 지...","(1) 연령: 18세 미만의 취학 아동 ( 초등학교, 중학교, 고등학교 재학 ) 및...","18세 미만의 취학 아동 및 학교밖 청소년 지원유형은 아동급식카드, 지역아동센터지원..."


In [2]:
import pandas as pd

# 파일 경로 설정
base_path = r"C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_정리.xlsx"
merge_path = r"C:\Users\hyunj\Women-s-Government-Aid\UI\정책큐레이션_통합데이터_v1.0.xlsx"

# 엑셀 파일 불러오기
base_df = pd.read_excel(base_path)
merge_df = pd.read_excel(merge_path)

# 병합 기준 열 설정 (1~4열: 인덱스 기준 0~3)
key_columns = base_df.columns[:4].tolist()

# 병합 수행 (base_df 기준으로 left join)
merged_df = pd.merge(base_df, merge_df, on=key_columns, how='left')

# 결과 저장
output_path = r"C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_병합.xlsx"
merged_df.to_excel(output_path, index=False)

print("병합 완료! 결과 파일 저장 위치:", output_path)


병합 완료! 결과 파일 저장 위치: C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_병합.xlsx


In [3]:
import pandas as pd

# 파일 불러오기
file_path = r"C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_병합.xlsx"
df = pd.read_excel(file_path)

# # 1. '_x'로 끝나는 열 삭제
# df.drop(columns=[col for col in df.columns if col.endswith('_x')], inplace=True)

# 2. '_y'로 끝나는 열 이름에서 '_y' 제거
df.rename(columns={col: col.replace('_x', '') for col in df.columns if col.endswith('_x')}, inplace=True)

# 결과 저장
output_path = r"C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_병합.xlsx"
df.to_excel(output_path, index=False)

print("열 정리 완료! 저장 위치:", output_path)


열 정리 완료! 저장 위치: C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_병합.xlsx


In [4]:
import pandas as pd

# 파일 경로
path_merged = r"C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_병합.xlsx"
path_organized = r"C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_정리.xlsx"

# 파일 불러오기
df_merged = pd.read_excel(path_merged)
df_organized = pd.read_excel(path_organized)

# 비교 기준 열
key_cols = ["대상유형", "지역", "제목", "detail_url", "신청기간", "신청방법", "접수기관", "지원형태", "지원대상", "지원내용"]

# 문자열 정제 함수
def clean_str(x):
    if pd.isna(x):
        return ""
    return str(x).strip().lower().replace("\n", "").replace("\r", "").replace(" ", "")

# 기준 열 정제
for col in key_cols:
    df_merged[col] = df_merged[col].apply(clean_str)
    df_organized[col] = df_organized[col].apply(clean_str)

# 병합 기준으로 row_key 생성
df_merged["row_key"] = df_merged[key_cols].agg("|".join, axis=1)
df_organized["row_key"] = df_organized[key_cols].agg("|".join, axis=1)

# 병합 파일 기준으로 정리 파일과 완전히 같은 행 제거
merged_only_diff = df_merged[~df_merged["row_key"].isin(df_organized["row_key"])]

# 결과 출력
print(merged_only_diff.drop(columns=["row_key"]))


Empty DataFrame
Columns: [대상유형, 지역, 제목, detail_url, 신청기간, 신청방법, 접수기관, 지원형태, 지원대상, 지원내용, 문의처, 기타, 카테고리_분류, 지원형태_분류, 지원대상_원문, 지원대상_초벌요약, 지원내용_원문, 지원내용_초벌요약, 신청기간_y, 신청방법_y, 접수기관_y, 지원형태_y, 지원대상_y, 지원내용_y, 문의처_y, 기타_y, 카테고리_분류_y, 지원형태_분류_y, 지원대상_원문_y, 지원대상_초벌요약_y, 지원내용_원문_y, 지원내용_초벌요약_y, age_eff_ranges, age_has_rule]
Index: []

[0 rows x 34 columns]


In [5]:
import pandas as pd

# 병합 파일 경로
PATH_MERGED = r"C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_병합.xlsx"

# 기준 열 정의
KEY_COLS = ["대상유형", "지역", "제목", "detail_url", "신청기간", "신청방법", "접수기관", "지원형태", "지원대상", "지원내용"]

# 파일 불러오기
df = pd.read_excel(PATH_MERGED)

# 문자열 정제 함수
def clean_str(x):
    if pd.isna(x):
        return ""
    return str(x).strip().lower().replace("\n", "").replace("\r", "").replace(" ", "")

# 기준 열 정제
for col in KEY_COLS:
    if col in df.columns:
        df[col] = df[col].apply(clean_str)

# row_key 생성
df["row_key"] = df[KEY_COLS].agg("|".join, axis=1)

# 중복 제거 (첫 번째 행만 남김)
df_unique = df.drop_duplicates(subset=["row_key"], keep="first").drop(columns=["row_key"])

# 결과 저장 (선택)
df_unique.to_excel("여성맞춤정책_요약_병합_최종.xlsx", index=False)

# 결과 확인
print(f"원래 행 수: {len(df)}")
print(f"유니크 행 수: {len(df_unique)}")


원래 행 수: 1308
유니크 행 수: 1292
